#### Quick Dash (dashit)

```python
@deepgreen.api
def magic():
    return 1
```

Spins up a server `if __name__ == "__main__"` else 

## My Function

In [1]:
from typing import Callable, Union, Any

In [2]:
from typing import List, Callable, Tuple, Dict

from functools import partial
import inspect
import pandas as pd

import flask
    
import dash
import dash_html_components as html
import dash_core_components as dcc

In [10]:
def magic(x:int,y:int,z, option1:str = "nothing", option2:Dict[str,Any] = "another", **kwargs):
    """My Function Does Everything
    
    First, it sets it up.
    
    Then, it knocks it down.
    
    """
    i,j,k = kwargs.pop("i",""), kwargs.pop("j",""), kwargs.pop("k","")
    return f"{x},{y},{z} -> magic <- {option1} {option2} ||| {i},{j},{k}"

In [11]:
import inspect
inspect.getfullargspec(magic)

FullArgSpec(args=['x', 'y', 'z', 'option1', 'option2'], varargs=None, varkw='kwargs', defaults=('nothing', 'another'), kwonlyargs=[], kwonlydefaults=None, annotations={'x': <class 'int'>, 'y': <class 'int'>, 'option1': <class 'str'>, 'option2': typing.Dict[str, typing.Any]})

In [24]:
types = inspect.getfullargspec(magic)
str(types.annotations['option2']).replace("typing.","")

'Dict[str, Any]'

# TODO: 

* named args should be expected as URL params .. that just makes sense

In [5]:
def f(x, y=1, **kwargs):
    return f"{x},{y}"

In [6]:
def g(x, y=2):
    a ** 2
    return f"g{x},{y}"

In [29]:
from typing import List, Callable, Union, Tuple

from functools import partial
import inspect
import pandas as pd

import flask


import dash
import dash_html_components as html
import dash_core_components as dcc

class DashitError(Exception):
    pass

def generate_rule(f: Callable, app: Union[dash.Dash, flask.Flask]) -> str:
    """
    Generate Flask rule to map URL positional and query params to function inputs
    """

    # Pull base url from either Flask or Dash app config.
    APP_BASE = app.config.get("APPLICATION_ROOT") or app.config.get("url_base_pathname")
    ENDPOINT = f.__name__

    positional, _ = parse_args(f)
    url_args = "".join([f"/<{arg}>" for arg in positional])

    rule = f"{APP_BASE}{ENDPOINT}{url_args}"
    return rule


def parse_args(f: Callable) -> Tuple[List[str], List[str]]:

    argspec = inspect.getfullargspec(f)
    args = argspec.args
    defaults = argspec.defaults
    positional, named = (
        args[: -len(defaults) if defaults else None],
        args[-len(defaults) if defaults else None :],
    )

    return positional, named


def whats_the_url(
    func: Callable, app: Union[dash.Dash, flask.Flask], *args, **kwargs
) -> str:
    """Generate the url to GET the function once you `dashit`"""
    
    signature = inspect.signature(func)
    call_errors = ["missing a required argument", "unexpected keyword argument"]
    try:
        arguments = signature.bind(*args, **kwargs).arguments
    except TypeError as e:
        if any([error in str(e) for error in call_errors]):
            msg = f"{e}. {func.__name__} is called like {func.__name__}{signature}"
            raise DashitError(msg) from e
        else:
            raise
            
    args = arguments.pop("args", [])
    kwargs = arguments.pop("kwargs", {})
    url = rule = generate_rule(func, app)
    print(args, kwargs, rule)
    # postional args
    for arg, val in arguments.items():
        url = url.replace(f"<{arg}>", str(val))
    # query params
    url += "?"
    url += "&".join([f"{keyword}={str(val)}" for keyword, val in kwargs.items()])
    url = url.strip("?")

    return url

def eval_params() -> Dict[str,Any]:
    """Attempt to evaluate dictionary values as python code, else return them as is"""
    import ast

    kwargs = {}
    for k, v in flask.request.args.to_dict(flat=True).items():
        try:
            kwargs[k] = ast.literal_eval(v)
        except:
            kwargs[k] = v
    return kwargs


def inject_flask_params_as_kwargs(func, **kwargs):
    """Flask converts the url variables into kwargs. We pass those back to our function as args. We also want to inject as kwargs the optional URL parameters"""
    positional = kwargs

    params = eval_params()  # &var1=["this"]&another=true -> {"var":["this"],"another":"true"}

    call_errors = ["missing a required argument", "unexpected keyword argument"]

    try:
        value = func(**positional, **params)

    except TypeError as e:
        if any([error in str(e) for error in call_errors]):
            signature = inspect.signature(func)
            msg = f"{func.__name__} is called like {func.__name__}{signature}. {e}."
            raise DashitError(msg) from e
        else:
            raise

    return value

def add_rule(app: Union[flask.Flask, dash.Dash], f: Callable):
    """
    Register function as Flask route. Positional and named arguments are all required. 
    In order to handle optional arguments, use **kwargs. *args, is right out.
    """
    rule = generate_rule(f, app)
    
    server = app if isinstance(app, flask.Flask) else app.server 
    server.add_url_rule(
        rule, endpoint=f.__name__, view_func=partial(_all_the_small_things, func=f)
    )
    f.url = partial(whats_the_url, f, server)
    return rule

def handle_wacky_types(thing):
    if isinstance(thing, pd.DataFrame):
        thing = thing.to_json(orient="table")
    return thing

def _all_the_small_things(func, **kwargs):
    response = inject_flask_params_as_kwargs(func, **kwargs)
    response = handle_wacky_types(response)
    return response

def dashit(functions: List[Callable], appname: str):
    """
    Create a QUICK Dash app exposing your functions as API endpoints!
    
    endpoint url:  
    
    All positional arguments will be required in the url. Named and keyword args are passed as query params.
    """

    app = dash.Dash(__name__, url_base_pathname=f"/{appname}/")    

    new_routes = [
        {
            "name": func.__name__,
            "docstring": inspect.cleandoc(func.__doc__) if func.__doc__ else "",
            "endpoint": add_rule(app, func),
        }
        for func in functions
    ]

    def generate_endpoint_html(route):
        return (
            dcc.Markdown(
                [f"""**{route["name"]}:** [{route["endpoint"]}]({route["endpoint"]})"""]
            ),
            html.Blockquote([dcc.Markdown([route["docstring"]])]),
        )

    flatten = lambda l: [item for sublist in l for item in sublist]

    app.layout = html.Div(
        [
            html.H1([appname]),
            html.H3(
                ["DASHIT: Functions Are APIs Now."]
            ),  # some explanation on how the app works. Use function docstring.
            html.Div(flatten([generate_endpoint_html(route) for route in new_routes])),
        ]
    )
    print(new_routes)

    return app


In [30]:
app = dashit([f,g], "test")

[{'name': 'f', 'docstring': '', 'endpoint': '/test/f/<x>'}, {'name': 'g', 'docstring': '', 'endpoint': '/test/g/<x>'}]


In [31]:
whats_the_url(g,app,1, y=30)

[] {} /test/g/<x>


'/test/g/1'

In [25]:
def h(x):
    return x

In [26]:
inspect.getfullargspec(h)

FullArgSpec(args=['x'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [27]:
add_rule(app, h)

'/test/h/<x>'

In [28]:
app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Mar/2020 13:53:20] "GET /test/h HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2020 13:53:21] "GET /test/_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2020 13:53:21] "GET /test/_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2020 13:53:28] "GET /test/h HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2020 13:53:28] "GET /test/_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2020 13:53:28] "GET /test/_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2020 13:53:28] "GET /test/_favicon.ico?v=1.9.1 HTTP/1.1" 200 -
